In [11]:
import affine, concurrent, cStringIO, glob, IPython, json, os, PIL, rasterio, shapely, sys, thread, time, traceback

#!conda install -y -c conda-forge geopandas
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

#!conda install -y rasterio
import rasterio
import rasterio.features

%matplotlib inline

In [12]:
class SimpleProcessPoolExecutor(concurrent.futures.ProcessPoolExecutor):
    def __init__(self, max_workers):
        super(SimpleProcessPoolExecutor, self).__init__(max_workers=max_workers)
        self.futures = []
        
    def submit(self, fn, *args, **kwargs):
        future = super(SimpleProcessPoolExecutor, self).submit(fn, *args, **kwargs)
        self.futures.append(future)
        return future
    
    def get_futures(self):
        return self.futures

    def shutdown(self):
        exception_count = 0
        results = []
        for completed in concurrent.futures.as_completed(self.futures):
            try:
                results.append(completed.result())
            except Exception as e:
                exception_count += 1
                sys.stderr.write(
                    'Exception caught in SimpleProcessPoolExecutor.shutdown.  Continuing until all are finished.\n' +
                    'Exception follows:\n' +
                    traceback.format_exc())
        super(SimpleProcessPoolExecutor, self).shutdown()
        if exception_count:
            raise Exception('SimpleProcessPoolExecutor failed: %d of %d raised exception' % (exception_count, len(self.futures)))
        print 'SimpleProcessPoolExecutor succeeded: all %d jobs completed' % (len(self.futures))
        return results

class Stopwatch:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        self.start = time.time()
    def __exit__(self, type, value, traceback):
        sys.stdout.write('%s took %.1f seconds\n' % (self.name, time.time() - self.start))


In [ ]:
geojsons = sorted(glob.glob('species_geojsons/*.geojson'))
print '%d species geojsons' % len(geojsons)

pngs = set(glob.glob('species_geojsons/*.png'))
print '%d species pngs' % len(pngs)

def rasterize(geojson, png):
    data = open(geojson).read()
    geom = json.loads(data)

    pixels_per_degree = 56

    transform = affine.Affine(1.0 / pixels_per_degree, 0,  -180,
                              0, -1.0 / pixels_per_degree, 90)

    with Stopwatch('%s from %d KB json' % (png, len(data)/1024)):
        i=rasterio.features.rasterize([geom],
                            all_touched=True,
                            out_shape=(180 * pixels_per_degree, 360 * pixels_per_degree),
                            transform=transform)
        tmp = png + '.%d.%d.png' % (thread.get_ident(), os.getpid())
        PIL.Image.fromarray(i*255).save(tmp)
        os.rename(tmp, png)

exe = SimpleProcessPoolExecutor(max_workers=25)

for geojson in geojsons:
    png = os.path.splitext(geojson)[0] + '.png'
    if not png in pngs:
        exe.submit(rasterize, geojson, png)

exe.shutdown()
None

35704 species geojsons
20375 species pngs
species_geojsons/ACTINOPTERYGII_ACIPENSERIFORMES_ACIPENSERIDAE_Scaphirhynchus_albus.png from 998 KB json took 9.6 seconds
species_geojsons/ACTINOPTERYGII_ACIPENSERIFORMES_ACIPENSERIDAE_Acipenser_brevirostrum.png from 1009 KB json took 12.0 seconds
species_geojsons/ACTINOPTERYGII_ALBULIFORMES_ALBULIDAE_Albula_koreana.png from 34 KB json took 11.2 seconds
species_geojsons/ACTINOPTERYGII_ACIPENSERIFORMES_ACIPENSERIDAE_Scaphirhynchus_platorynchus.png from 2234 KB json took 11.8 seconds
species_geojsons/ACTINOPTERYGII_ACIPENSERIFORMES_POLYODONTIDAE_Polyodon_spathula.png from 3134 KB json took 13.5 seconds
species_geojsons/ACTINOPTERYGII_ALBULIFORMES_ALBULIDAE_Albula_esuncula.png from 1943 KB json took 17.9 seconds
species_geojsons/ACTINOPTERYGII_ALBULIFORMES_ALBULIDAE_Albula_virgata.png from 77 KB json took 14.1 seconds
species_geojsons/ACTINOPTERYGII_ACIPENSERIFORMES_ACIPENSERIDAE_Scaphirhynchus_suttkusi.png from 59 KB json took 14.7 seconds
specie

species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Alestopetersius_hilgendorfi.png from 664 KB json took 19.8 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Alestopetersius_leopoldianus.png from 434 KB json took 20.2 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Alestopetersius_caudalis.png from 1030 KB json took 21.0 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Alestopetersius_tumbensis.png from 251 KB json took 26.7 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Bathyaethiops_caudomaculatus.png from 620 KB json took 29.3 seconds
species_geojsons/ACTINOPTERYGII_BELONIFORMES_HEMIRAMPHIDAE_Hemirhamphodon_pogonognathus.png from 1844 KB json took 88.4 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Brachypetersius_notospilus.png from 42 KB json took 29.2 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Brycinus_abeli.png from 20 KB json took 31.5 seconds
species_geojsons/ACTINOPTERYG

species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Brycinus_nurse.png from 4145 KB json took 106.4 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Phenacogrammus_bleheri.png from 15 KB json took 25.4 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Phenacogrammus_aurantiacus.png from 968 KB json took 32.0 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Phenacogrammus_major.png from 168 KB json took 23.6 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Phenacogrammus_interruptus.png from 929 KB json took 24.7 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Phenacogrammus_stigmatura.png from 13 KB json took 21.6 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Micralestes_argyrotaenia.png from 34 KB json took 64.1 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_Phenacogrammus_polli.png from 393 KB json took 25.8 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_ALESTIDAE_

species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Argopleura_conventus.png from 5 KB json took 76.1 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Bryconamericus_bolivianus.png from 128 KB json took 21.4 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Bryconamericus_arilepis.png from 11 KB json took 23.2 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Bryconamericus_caucanus.png from 144 KB json took 22.9 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Astyanax_chaparae.png from 18 KB json took 68.8 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Bryconamericus_diaphanus.png from 132 KB json took 20.3 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Bryconamericus_foncensis.png from 11 KB json took 21.5 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Astyanax_caucanus.png from 85 KB json took 71.1 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_

species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Grundulus_bogotensis.png from 16 KB json took 19.0 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Grundulus_cochae.png from 1 KB json took 21.7 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Hemibrycon_brevispini.png from 3 KB json took 19.5 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Hemibrycon_carrilloi.png from 8 KB json took 17.5 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Creagrutus_flavescens.png from 208 KB json took 71.0 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Hemibrycon_dentatus.png from 119 KB json took 17.3 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Hemibrycon_divisorensis.png from 8 KB json took 17.2 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Hemibrycon_helleri.png from 19 KB json took 21.0 seconds
species_geojsons/ACTINOPTERYGII_CHARACIFORMES_CHARACIDAE_Hemibrycon_huamboni

...

species_geojsons/MALACOSTRACA_DECAPODA_POTAMONAUTIDAE_Sudanonautes_floweri.png from 1410 KB json took 12.1 seconds
species_geojsons/MALACOSTRACA_DECAPODA_POTAMONAUTIDAE_Sudanonautes_kagoroensis.png from 54 KB json took 11.4 seconds
species_geojsons/MALACOSTRACA_DECAPODA_POTAMONAUTIDAE_Sudanonautes_nigeria.png from 38 KB json took 11.4 seconds
species_geojsons/MALACOSTRACA_DECAPODA_POTAMONAUTIDAE_Sudanonautes_monodi.png from 495 KB json took 11.8 seconds
species_geojsons/MALACOSTRACA_DECAPODA_POTAMONAUTIDAE_Sudanonautes_sangha.png from 8 KB json took 11.3 seconds
species_geojsons/MALACOSTRACA_DECAPODA_POTAMONAUTIDAE_Potamonautes_didieri.png from 37 KB json took 85.5 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Allacanthos_pittieri.png from 37 KB json took 11.6 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Anchlidon_agrestis.png from 106 KB json took 10.9 seconds
species_geojsons/MALACOSTRACA_DECAPODA_POTAMONAUTIDAE_Sudanonautes_orthostylis.png fr

species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Hypolobocera_meineli.png from 156 KB json took 11.9 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Hypolobocera_mindonensis.png from 51 KB json took 12.4 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Hypolobocera_muisnensis.png from 21 KB json took 13.4 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Hypolobocera_mutisi.png from 32 KB json took 13.3 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Hypolobocera_noanamensis.png from 54 KB json took 13.8 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Hypolobocera_murindensis.png from 178 KB json took 14.7 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Hypolobocera_orcesi.png from 69 KB json took 14.3 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Hypolobocera_peruviana.png from 621 KB json took 13.6 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOT

species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Neostrengeria_sketi.png from 37 KB json took 13.5 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Neostrengeria_tonensis.png from 68 KB json took 12.7 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Odontothelphusa_lacandona.png from 283 KB json took 13.8 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Odontothelphusa_lacanjaensis.png from 283 KB json took 13.6 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Odontothelphusa_maxillipes.png from 260 KB json took 12.8 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Odontothelphusa_monodontis.png from 120 KB json took 13.4 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Odontothelphusa_palenquensis.png from 285 KB json took 16.0 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Odontothelphusa_toninae.png from 58 KB json took 14.8 seconds
species_geojsons/MALAC

species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Ptychophallus_osaensis.png from 26 KB json took 16.3 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Ptychophallus_paraxanthusi.png from 118 KB json took 15.2 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Ptychophallus_tristani.png from 50 KB json took 15.7 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Ptychophallus_uncinatus.png from 26 KB json took 13.1 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Raddaus_mertensi.png from 51 KB json took 12.1 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Raddaus_orestrius.png from 71 KB json took 11.1 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Raddaus_bocourti.png from 389 KB json took 14.3 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Raddaus_tuberculatus.png from 45 KB json took 11.3 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Ro

species_geojsons/MALACOSTRACA_DECAPODA_TRICHODACTYLIDAE_Rodriguezia_villalobosi.png from 82 KB json took 13.6 seconds
species_geojsons/MALACOSTRACA_DECAPODA_PSEUDOTHELPHUSIDAE_Ptychophallus_tumimanus.png from 53 KB json took 94.6 seconds
species_geojsons/MALACOSTRACA_DECAPODA_TRICHODACTYLIDAE_Rotundovaldivia_latidens.png from 627 KB json took 14.7 seconds
species_geojsons/MALACOSTRACA_DECAPODA_TRICHODACTYLIDAE_Rodriguezia_mensabak.png from 19 KB json took 17.1 seconds
species_geojsons/MALACOSTRACA_DECAPODA_TRICHODACTYLIDAE_Poppiana_dentata.png from 1685 KB json took 19.0 seconds
species_geojsons/MALACOSTRACA_DECAPODA_TRICHODACTYLIDAE_Sylviocarcinus_australis.png from 1231 KB json took 16.2 seconds
species_geojsons/MALACOSTRACA_DECAPODA_TRICHODACTYLIDAE_Sylviocarcinus_devillei.png from 1690 KB json took 19.1 seconds
species_geojsons/MALACOSTRACA_DECAPODA_TRICHODACTYLIDAE_Sylviocarcinus_piriformis.png from 477 KB json took 16.4 seconds
species_geojsons/MALACOSTRACA_DECAPODA_TRICHODACTYLI

In [6]:
rasterize('species_geojsons/MAMMALIA_CARNIVORA_URSIDAE_Ursus_americanus.geojson', 'species_geojsons/MAMMALIA_CARNIVORA_URSIDAE_Ursus_americanus.png')
PIL.Image.open('species_geojsons/MAMMALIA_CARNIVORA_URSIDAE_Ursus_americanus.png')

species_geojsons/MAMMALIA_CARNIVORA_URSIDAE_Ursus_americanus.png from 5166 KB json took 7.7 seconds


/home/rsargent/anaconda2/lib/python2.7/site-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (203212800 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [75]:
PIL.Image.open('species_geojsons/MAMMALIA_CARNIVORA_URSIDAE_Ursus_americanus.png')